<h1>Pipeline</h1>

In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline # this is research only

Good to wrap this in a function to set up more complex pipelines, for now this is just an empty pipeline.

In [2]:
def make_pipeline():
    return Pipeline()


The output of an empty pipeline is a DataFrame with no columns. In this example, our pipeline has an index made up of all 8000+ securities (only 5 rows displayed in the image) for May 5th, 2015, but doesn't have any columns.

In [3]:
my_pipe = make_pipeline()

# running the pipeline returns a df, double index data and secs
result = run_pipeline(my_pipe, '2015-05-05', '2015-05-05')
result.head() 

#currently empty pipeline, all indexes and no columns

Empty DataFrame
Columns: []
Index: [(2015-05-05 00:00:00+00:00, Equity(2 [HWM])), (2015-05-05 00:00:00+00:00, Equity(21 [AAME])), (2015-05-05 00:00:00+00:00, Equity(24 [AAPL])), (2015-05-05 00:00:00+00:00, Equity(25 [HWM_PR])), (2015-05-05 00:00:00+00:00, Equity(31 [ABAX]))]

<h1>Factors</h1> <br>
Check the parameters required for the factor to be used
<br>
<a href = "https://www.quantopian.com/docs/api-reference/pipeline-api-reference#pipeline-quickref-factors">Open Link in new tab</a>

In [4]:
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage

# each factor needs to be instantiated with parameters to be actually used
# to check params visit link above

def make_pipeline():
    
    # this is just instantiating the function, not doing any computations
    mean_close_10 = SimpleMovingAverage(
        inputs = [USEquityPricing.close], # BOUNDColumn indicates the data type
        window_length = 10
    )
    
    # attach to pipeline - this will do the computations the given col name
    pipe = Pipeline(
        columns = {
            '10_day_mean_close': mean_close_10
        }
    )
    
    return pipe

The DataFrame has a MultiIndex where the first level is a datetime representing the date of the computation and the second level is an Equity object corresponding to the security. For example, the first row in the above DataFrame (`2015-05-05 00:00:00+00:00`, `Equity(2 [AA])`) contains the result of our mean_close_10 factor for AA on May 5th, 2015.

In [5]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result.head()

10_day_mean_close
2015-05-05 00:00:00+00:00 Equity(2 [HWM])                13.5595
                          Equity(21 [AAME])               3.9625
                          Equity(24 [AAPL])             129.0257
                          Equity(25 [HWM_PR])            88.3625
                          Equity(31 [ABAX])              61.9209

# Latest - inbuilt factor
The Latest factor gets the most recent value of a given data column. This factor is common enough that it is instantiated differently from other factors. The best way to get the latest value of a data column is by getting its .latest attribute. 

In [6]:
def make_pipeline():
    
    # this is just instantiating the function, not doing any computations
    mean_close_10 = SimpleMovingAverage(
        inputs = [USEquityPricing.close], # BOUNDColumn indicates the data type
        window_length = 10
    )
    
    # LATEST IS INSTANTIATED DIFFERENTLY TO OTHER FACTORS, can even return other things than factors!
    latest_close = USEquityPricing.close.latest
    
    # attach to pipeline - this will do the computations the given col name
    pipe = Pipeline(
        columns = {
            '10_day_mean_close': mean_close_10,
            'latest_closing_price': latest_close
        }
    )
    
    return pipe

In [7]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result.head()

10_day_mean_close  \
2015-05-05 00:00:00+00:00 Equity(2 [HWM])                13.5595   
                          Equity(21 [AAME])               3.9625   
                          Equity(24 [AAPL])             129.0257   
                          Equity(25 [HWM_PR])            88.3625   
                          Equity(31 [ABAX])              61.9209   

                                               latest_closing_price  
2015-05-05 00:00:00+00:00 Equity(2 [HWM])                    14.015  
                          Equity(21 [AAME])                     NaN  
                          Equity(24 [AAPL])                 128.699  
                          Equity(25 [HWM_PR])                   NaN  
                          Equity(31 [ABAX])                  55.030

Some factors have default inputs that should never be changed. For example the VWAP built-in factor is always calculated from USEquityPricing.close and USEquityPricing.volume. When a factor is always calculated from the same BoundColumns, we can call the constructor without specifying inputs.

In [8]:
from quantopian.pipeline.factors import VWAP
vwap = VWAP(window_length=10)

# Combining Factors

Factors can be combined, both with other Factors and with scalar values, via any of the builtin mathematical operators (+, -, *, etc). <br><br>
f1 = SomeFactor(...)<br>
f2 = SomeOtherFactor(...)<br>
average = (f1 + f2) / 2.0

In [9]:
def make_pipeline():
    
    # this is just instantiating the function, not doing any computations
    mean_close_10 = SimpleMovingAverage(
        inputs = [USEquityPricing.close], # BOUNDColumn indicates the data type
        window_length = 10
    )
    
    mean_close_30 = SimpleMovingAverage(
        inputs = [USEquityPricing.close], # BOUNDColumn indicates the data type
        window_length = 30
    )
    
    # difining a higher order factor that is NOT YET COMPUTED
    # static type of factor
    percent_difference = (mean_close_10 - mean_close_30)/ mean_close_30
    
    # LATEST IS INSTANTIATED DIFFERENTLY TO OTHER FACTORS, can even return other things than factors!
    latest_close = USEquityPricing.close.latest
    
    # attach to pipeline - this will do the computations the given col name
    pipe = Pipeline(
        columns = {
            '10_day_mean_close': mean_close_10,
            '30_day_mean_close': mean_close_30,
            'percent_difference': percent_difference,
            'latest_closing_price': latest_close
        }
    )
    
    return pipe

In [10]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result.head()

10_day_mean_close  \
2015-05-05 00:00:00+00:00 Equity(2 [HWM])                13.5595   
                          Equity(21 [AAME])               3.9625   
                          Equity(24 [AAPL])             129.0257   
                          Equity(25 [HWM_PR])            88.3625   
                          Equity(31 [ABAX])              61.9209   

                                               30_day_mean_close  \
2015-05-05 00:00:00+00:00 Equity(2 [HWM])              13.320067   
                          Equity(21 [AAME])             3.971735   
                          Equity(24 [AAPL])           126.880733   
                          Equity(25 [HWM_PR])          86.498944   
                          Equity(31 [ABAX])            63.161300   

                                               latest_closing_price  \
2015-05-05 00:00:00+00:00 Equity(2 [HWM])                    14.015   
                          Equity(21 [AAME])                     NaN   
                          Equity(24 [AAPL])                 128.699   
                          Equity(25 [HWM_PR])                   NaN   
                          Equity(31 [ABAX])                  55.030   

                                               percent_difference  
2015-05-05 00:00:00+00:00 Equity(2 [HWM])                0.017975  
                          Equity(21 [AAME])             -0.002325  
                          Equity(24 [AAPL])              0.016905  
                          Equity(25 [HWM_PR])            0.021544  
                          Equity(31 [ABAX])             -0.019639

# Filters

f (asset, timestamp) -> boolean

Use when narrowing down set of securities included in computation or in the final output of a pipeline.

Comparison operators on Factors and Classifiers produce Filters

In [11]:
last_close_price = USEquityPricing.close.latest
filter_close_price = last_close_price > 20

In [12]:
# we can alternatively actually produce these filters sa a column which would
# simplify the code in rebalance quite a bit

# here instead of checking these two columns when iterating over the securities
# we could simply record the signals as a column in itself
mean_crossover_filter = mean_close_10 < mean_close_30

NameError: name 'mean_close_10' is not defined

#    

Various methods of the Factor and Classifier classes return Filters. The Factor.top(n) method produces a Filter that returns True for the top n securities of a given factor each day. The following example produces a filter that returns True for exactly 200 securities every day, indicating that those securities were in the top 200 by last close price across all known securities.

<b>Link of factor methodsthat return filters (inbuilt):

METHODS THAT CREATE FACTORS

FACTORS THAT CREATE OTHER FACTORS (LIKE LINEAR REGRESSION AND OTHER STUFF BUILT IN!) </b>

https://www.quantopian.com/docs/api-reference/pipeline-api-reference#pipeline-quickref-factor-methods


In [ ]:
last_close_price = USEquityPricing.close.latest
top_close_price_filter = last_close_price.top(200)

Creating a filter that returns True if a security's 30-day average dollar volume is above $10,000,000. To do this, we'll first need to create an AverageDollarVolume factor to compute the 30-day average dollar volume.

In [ ]:
from quantopian.pipeline.factors import AverageDollarVolume, SimpleMovingAverage

In [ ]:
def make_pipeline():
    
    # this factor by default uses equitypricing.close and
    # equitypricing.volume as inputs so they don't need to be specified
    dollar_volume = AverageDollarVolume(window_length = 30)
    filter_volume = dollar_volume > 10**7
    
    # this is just instantiating the function, not doing any computations
    mean_close_10 = SimpleMovingAverage(
        inputs = [USEquityPricing.close], # BOUNDColumn indicates the data type
        window_length = 10
    )
    
    mean_close_30 = SimpleMovingAverage(
        inputs = [USEquityPricing.close], # BOUNDColumn indicates the data type
        window_length = 30
    )
    
    # difining a higher order factor that is NOT YET COMPUTED
    # static type of factor
    percent_difference = (mean_close_10 - mean_close_30)/ mean_close_30

    # attach to pipeline - this will do the computations the given col name
    pipe = Pipeline(
        columns = {
            'percent_difference': percent_difference,
            'high_dollar_volume':filter_volume
        }
    )
    
    return pipe

In [ ]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result.head()

So far we have learnt to produce all of these filters and factors as columns themselves, and quantopian by default produces results for each security for EACH DAY. We may only want a subset of these securities to actually even be considered. How do?

# Apply a Screen

We can tell our Pipeline to ignore securities for which a filter produces False by passing that filter to our Pipeline via the screen keyword.

To screen our pipeline output for securities with a 30-day average dollar volume greater than $10,000,000, we can simply pass our high_dollar_volume filter as the screen argument.


In [ ]:
def make_pipeline():
    
    # this factor by default uses equitypricing.close and
    # equitypricing.volume as inputs so they don't need to be specified
    dollar_volume = AverageDollarVolume(window_length = 30)
    filter_volume = dollar_volume > 10**7
    
    # this is just instantiating the function, not doing any computations
    mean_close_10 = SimpleMovingAverage(
        inputs = [USEquityPricing.close], # BOUNDColumn indicates the data type
        window_length = 10
    )
    
    mean_close_30 = SimpleMovingAverage(
        inputs = [USEquityPricing.close], # BOUNDColumn indicates the data type
        window_length = 30
    )
    
    # difining a higher order factor that is NOT YET COMPUTED
    # static type of factor
    percent_difference = (mean_close_10 - mean_close_30)/ mean_close_30

    # attach to pipeline - this will do the computations the given col name
    pipe = Pipeline(
        columns = {
            'percent_difference': percent_difference,
        },
        screen = filter_volume
    )
    
    return pipe

In [ ]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result.head()
print('Number of securities that passed the filter: %d' % len(result))

# Higher order filters

Filters can also be inverted using "~"

They can be combined as well into a new filter using any boolean operators.

New filter => all_volume = high_dollar_vol & low_dollar_vol

In [ ]:
low_dollar_vol = ~ filter_volume # filter_volume is high_dollar_vol filter

# Masking

Sometimes we want to ignore certain assets when computing pipeline expresssions. There are two common cases where ignoring assets is useful:

1. We want to compute an expression that's computationally expensive, and we know we only care about results for certain assets. A common example of such an expensive expression is a Factor computing the coefficients of a regression (RollingLinearRegressionOfReturns).

1. We want to compute an expression that performs comparisons between assets, but we only want those comparisons to be performed against a subset of all assets. For example, we might want to use the top method of Factor to compute the top 200 assets by earnings yield, ignoring assets that don't meet some liquidity constraint.

To support these two use-cases, all Factors and many Factor methods can accept a mask argument, which must be a Filter indicating which assets to consider when computing.

<b><i><u> Masking is applied to the FACTOR itself, EXTREMELY efficient as this filters assets / dates BEFORE the computation is performed, where SCREEN computes everything, and filters AFTER. 
    
# So for the GOOD ALPHA, this will be the way to go.


In [ ]:
# Dollar volume factor
dollar_volume = AverageDollarVolume(window_length=30)

# High dollar volume filter
high_dollar_volume = (dollar_volume > 10000000)

# Average close price factors
mean_close_10 = SimpleMovingAverage(
    inputs=[USEquityPricing.close],
    window_length=10,
    mask=high_dollar_volume
)
mean_close_30 = SimpleMovingAverage(
    inputs=[USEquityPricing.close],
    window_length=30,
    mask=high_dollar_volume
)

# Relative difference factor
percent_difference = (mean_close_10 - mean_close_30) / mean_close_30

Applying the mask to SimpleMovingAverage restricts the average close price factors to a computation over the ~2000 securities passing the high_dollar_volume filter, as opposed to ~8000 without a mask. When we combine mean_close_10 and mean_close_30 to form percent_difference, the computation is performed on the same ~2000 securities.

# Masking Filters

Masks can be also be applied to methods that return filters like top, bottom, and percentile_between.

Masks are most useful when we want to apply a filter in the earlier steps of a combined computation. For example, suppose we want to get the 50 securities with the highest open price that are also in the top 10% of dollar volume. Suppose that we then want the 90th-100th percentile of these securities by close price. We can do this with the following:


In [ ]:
def make_pipeline():

    # Dollar volume factor
    dollar_volume = AverageDollarVolume(window_length=30)

    # High dollar volume filter
    high_dollar_volume = dollar_volume.percentile_between(90,100)
    
    # ONLY GET TOP 50 PRICES FOR HIGH DOLLAR VOLUME SECS
    # Top open securities filter (high dollar volume securities)
    top_open_price = USEquityPricing.open.latest.top(50, mask=high_dollar_volume)
    
    # ONLY GET TOP PERCENTILES FOR THE TOP_OPEN_PRICE (TAKES TOP 50 OF HIGH DOLLAR VOLUMES)
    # Top percentile close price filter (high dollar volume, top 50 open price)
    high_close_price = USEquityPricing.close.latest.percentile_between(90, 100, mask=top_open_price)
    
    
    # THINK OF THIS SORT OF MASKING AS ASSET FUNNELING
    return Pipeline(
        screen=high_close_price
    )

In [ ]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result

# CLASSIFIERS - SKIPPED AS I DON'T THINK WE NEED IT

# DATASETS AND STATIC TYPING IN THIS PLATFORM

Please read this one is super important <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
https://www.quantopian.com/tutorials/pipeline#lesson9

# CUSTOM FACTORS

https://www.quantopian.com/tutorials/pipeline#lesson10

Conceptually, a custom factor is identical to a built-in factor. It accepts inputs, window_length, and mask as constructor arguments, and returns a Factor object each day.

def compute(self, today, asset_ids, out, *inputs)

1. *inputs are M x N numpy arrays, where M is the window_length and N is the number of securities (usually around ~8000 unless a mask is provided). *inputs are trailing data windows. Note that there will be one M x N array for each BoundColumn provided in the factor's inputs list. The data type of each array will be the dtype of the corresponding BoundColumn.
1. out is an empty array of length N. out will be the output of our custom factor each day. The job of compute is to write output values into out.
1. asset_ids will be an integer array of length N containing security ids corresponding to the columns in our *inputs arrays.
1. today will be a pandas Timestamp representing the day for which compute is being called.

In [6]:
from quantopian.pipeline import CustomFactor, Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
import numpy

# we can create a std deviation factor (over a trailing window) 
# as it does not exist as a built-in


class StdDev(CustomFactor):
    
    def compute(self, today, asset_ids, out, values):
        
        # Calculates the column-wise standard deviation, ignoring NaNs
        out[:] = numpy.nanstd(values, axis=0)

https://numpy.org/doc/stable/reference/generated/numpy.nanstd.html

In [12]:
def make_pipeline():
    
    std_dev = StdDev(
        inputs = [USEquityPricing.close],
        window_length = 5
    )
    
    pipe = Pipeline(
        columns = {
            'std_dev': std_dev,
        }
    )
    
    return pipe
    
    

In [15]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result.head()

std_dev
2015-05-05 00:00:00+00:00 Equity(2 [HWM])      0.293428
                          Equity(21 [AAME])    0.004714
                          Equity(24 [AAPL])    1.737677
                          Equity(25 [HWM_PR])  0.275000
                          Equity(31 [ABAX])    4.402971

In [16]:
from scipy import stats
import numpy as np

def add_one(x):
    x[0] = x[0] + 1
    return x

# how does this even work, what is axis???
x = np.array([[1,2],[3,4]])
x = np.apply_along_axis(add_one, axis = 0, arr=x)
x



array([[2, 3],
       [3, 4]])

In [17]:
x = np.array([1,2,3,4])
y = np.array([4,3,2,1])
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
slope, intercept, r_value, p_value, std_err

(-1.0, 5.0, -1.0, 9.9999999999999979e-21, 0.0)

# Trying to first implement a standard regression myself, to see if the logic is right

- One available source here as rollinglinearRegOfReturns implementation
    - https://www.quantopian.com/posts/building-a-better-beta

- so here we potentially need to use 30 day returns instead of the daily returns since our algo is monthly
- 

# ROLLING REGRESSION OF RETURNS

In [11]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline import CustomFactor
from quantopian.pipeline.factors import DailyReturns, Returns, SimpleBeta, AverageDollarVolume, SimpleMovingAverage
import numpy as np
from scipy import stats


# here we need to find a way to regress this NOT on SPY inputs,
# but instead if we can transform the INPUTS variable to the 
# same structure as it is now, but instead of returns from SPY it is our portfolio
# our portoflio can be stored in the context object!!!!

class MyRollingLinearRegressionOfReturns(CustomFactor):
    outputs = ['alpha', 'beta', 'r_value', 'p_value', 'stderr', 'alpha2']
#     inputs = [DailyReturns(), DailyReturns()[symbols('SPY')]]
    inputs = [DailyReturns(), DailyReturns()[symbols('SPY')]]

    # list of inputs (*inputs) -> First is dependent, second is independent
    
    # Regression y = beta*x + a
    
    def compute(self, today, assets, out, dependent, independent):
        # think of these as the attributes of the factor computation object
        # each one of these can be called upon in a different column
        alpha = out.alpha
#         alpha2 = out.alpha #test
        beta = out.beta
        r_value = out.r_value
        p_value = out.p_value
        stderr = out.stderr
#         print(alpha)
#         print(dependent)
#         print(independent)
        
        # i in here is from the for loop below
        def regress(y, x):
            regr_results = stats.linregress(y=y, x=x)
            alpha[i] = regr_results.intercept
            beta[i] = regr_results.slope
            r_value[i] = regr_results.rvalue
            p_value[i] = abs(regr_results.pvalue)
            stderr[i] = regr_results.stderr

        for i in range(len(out)):
            y=dependent[:, i].ravel()
            x=independent.ravel()
            regress(y, x)
#             print(y, x)
            
#     [A B C D E] EQUAL
#     [0 0 0 0.5 -0.5]
    
            
def make_pipeline():
    
#    initial_portfolio = [30-day SPY return]  <- in initialise or somehting
#    new_array = initial_portfolio
#    new_array = new_array.append(ourPortfolio.return)
#    new_array.pop()

    regression = MyRollingLinearRegressionOfReturns(
        window_length = 10
    )
    
    significant_bad_alpha = regression.p_value < 0.05
    
    
    daily_returns = DailyReturns(window_length = 5)
    
    top_alphas = regression.alpha.top(1000)
    last_alphas = regression.alpha.bottom(1000)
    
    pipe = Pipeline(
        columns = {
            'beta': regression.beta,
            'alpha': regression.alpha*100,
            'daily_returns': daily_returns,
            'p-val': regression.p_value
#             'alpha2': regression.alpha2, #test
#             'daily_returns': DailyReturns()
        },
        screen = (top_alphas | last_alphas) & significant_bad_alpha
    )
#    in rebalance (called at end of month)
#     
    
    return pipe

In [12]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result

alpha      beta  \
2015-05-05 00:00:00+00:00 Equity(41 [ARCB])      -0.792533  3.141711   
                          Equity(53 [ABMD])      -0.794345  2.626321   
                          Equity(67 [ADSK])      -0.756424  3.062459   
                          Equity(110 [RAMP])     -0.799114  1.885867   
                          Equity(153 [AE])       -2.914037  4.517824   
                          Equity(273 [ALU])      -1.587838  4.374247   
                          Equity(301 [ALKS])     -0.914211  3.979062   
                          Equity(557 [ASGN])     -1.674617  4.948768   
                          Equity(617 [ATRO])     -0.811192  2.875352   
                          Equity(624 [ATW])       1.315270 -4.368670   
                          Equity(659 [AMAG])     -0.963465  4.827623   
                          Equity(915 [BKE])      -0.686680  1.507066   
                          Equity(935 [CNR])      -0.908398  1.814443   
                          Equity(1062 [BPOP])    -0.793454  1.505505   
                          Equity(1216 [CACC])     1.707754 -3.688789   
                          Equity(1315 [CBT])     -0.771691  3.653377   
                          Equity(1376 [CAH])     -0.725611  3.166115   
                          Equity(1460 [CGNX])    -0.833561  2.809091   
                          Equity(1675 [CMTL])    -0.712256  2.790629   
                          Equity(1751 [COHR])    -1.268622  2.534397   
                          Equity(1884 [CRVL])    -0.799225  2.254897   
                          Equity(1923 [CSPI])     0.467866 -4.843077   
                          Equity(2404 [EAT])     -0.724543  1.947276   
                          Equity(3129 [BGC])     -0.823288  3.048474   
                          Equity(3150 [INO])     -2.319181  7.209186   
                          Equity(3203 [GIB])     -0.721539  1.972919   
                          Equity(3241 [GLW])     -0.699589  2.007987   
                          Equity(3490 [PEAK])    -0.722535  1.469227   
                          Equity(3519 [HEI])     -0.726514  1.756665   
                          Equity(3801 [IDCC])     0.791826  1.288360   
...                                                    ...       ...   
                          Equity(46583 [AKBA])   -0.929786  3.834490   
                          Equity(46610 [KING])   -1.085871  1.483997   
                          Equity(46721 [LQ])      0.431598  0.886904   
                          Equity(46778 [IMUX])   -1.391591  5.399063   
                          Equity(46910 [SFBS])    0.511895  1.386152   
                          Equity(46932 [TBPH])   -1.363878  4.707544   
                          Equity(46951 [CTRE])   -0.791025  2.218895   
                          Equity(47146 [ARDX])   -1.778964  4.664228   
                          Equity(47169 [KITE])   -1.086346  5.622027   
                          Equity(47227 [GBIM])   -1.245617  6.329030   
                          Equity(47322 [ROKA])   -0.733783  3.514119   
                          Equity(47333 [TUBE])   -0.749377  2.049706   
                          Equity(47351 [NAME])   -1.222216  6.319095   
                          Equity(47383 [OCUL])   -1.963230  3.582296   
                          Equity(47414 [CTLT])   -0.679335  2.198570   
                          Equity(47422 [ADVM])   -2.756901  5.359788   
                          Equity(47432 [LOXO])   -1.269875  4.103429   
                          Equity(47465 [HMLP])    0.465181  2.191033   
                          Equity(47476 [GNBC])    1.274126  4.778150   
                          Equity(47826 [BTX_WS])  1.079866  7.464049   
                          Equity(48025 [NVRO])   -1.560076  5.416558   
                          Equity(48126 [HABT])    1.776284  5.200374   
                          Equity(48139 [CPHR])   -0.858368  5.087843   
                          Equity(48215 [QSR])     0.802513  3.222475   
                     

https://www.quantopian.com/posts/strange-behaviour-of-customfactor-calculating-slope-of-linear-regression

In [49]:
x =np.array( [[ 0.00308408],
 [-0.00387872],
 [-0.01011444],
 [ 0.0106975 ],
 [ 0.00313256]])
x.ravel()

array([ 0.00308408, -0.00387872, -0.01011444,  0.0106975 ,  0.00313256])

In [ ]:
x = DailyReturns()